# openai audio output

In [ ]:
import base64
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-audio-preview",
    modalities=["text", "audio"],
    audio={"voice": "alloy", "format": "wav"},
    messages=[
        {
            "role": "user",
            "content": "Is a golden retriever a good family dog?"
        }
    ]
)

print(completion.choices[0])

wav_bytes = base64.b64decode(completion.choices[0].message.audio.data)
with open("dog.wav", "wb") as f:
    f.write(wav_bytes)



In [ ]:
import base64
import streamlit as st
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Function to generate audio response
def generate_audio_response(messages):
    completion = client.chat.completions.create(
        model="gpt-4o-audio-preview",
        modalities=["text", "audio"],
        audio={"voice": "alloy", "format": "wav"},
        messages=messages
    )
    return completion.choices[0]

# Function to decode and save audio
def save_audio(audio_data, filename):
    wav_bytes = base64.b64decode(audio_data)
    with open(filename, "wb") as f:
        f.write(wav_bytes)

# Initialize conversation history
conversation_history = []

# Streamlit UI
st.title("Multi-turn Conversation Bot")

# User input
user_input = st.text_input("You: ", "")

if user_input:
    # Add user message to conversation history
    conversation_history.append({"role": "user", "content": user_input})

    # Generate response
    response = generate_audio_response(conversation_history)

    # Add assistant message to conversation history
    conversation_history.append(response.message)

    # Save and play audio
    audio_filename = f"{response.message.audio.id}.wav"
    save_audio(response.message.audio.data, audio_filename)
    st.audio(audio_filename, format="audio/wav", autoplay=True)

    # Display transcript
    st.write("Assistant: ", response.message.audio.transcript)

# Display conversation history
st.subheader("Conversation History")
for message in conversation_history:
    if isinstance(message, dict):  # User message
        role = "You"
        content = message["content"]
    else:  # Assistant message
        role = "Assistant"
        content = message.audio.transcript
    st.write(f"{role}: {content}")

# elevenlab


In [ ]:
!python -m pip install elevenlabs

In [7]:

import os
import uuid
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs

ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
client = ElevenLabs(
    api_key=ELEVENLABS_API_KEY,
)


def text_to_speech_file(text: str) -> str:
    # Calling the text_to_speech conversion API with detailed parameters
    response = client.text_to_speech.convert(
        voice_id="pNInz6obpgDQGcFmaJgB", # Adam pre-made voice
        output_format="mp3_22050_32",
        text=text,
        model_id="eleven_turbo_v2_5", # use the turbo model for low latency
        voice_settings=VoiceSettings(
            stability=0.0,
            similarity_boost=1.0,
            style=0.0,
            use_speaker_boost=True,
        ),
    )

    # uncomment the line below to play the audio back
    # play(response)

    # Generating a unique file name for the output MP3 file
    save_file_path = f"{uuid.uuid4()}.mp3"

    # Writing the audio to a file
    with open(save_file_path, "wb") as f:
        for chunk in response:
            if chunk:
                f.write(chunk)

    print(f"{save_file_path}: A new audio file was saved successfully!")

    # Return the path of the saved audio file
    return save_file_path

text_to_speech_file("Hello World")

ffe8e309-cb36-462a-a88d-fe7538a01d3a.mp3: A new audio file was saved successfully!


'ffe8e309-cb36-462a-a88d-fe7538a01d3a.mp3'